#### Install Dependencies

In [ ]:
!pip install transformers peft bitsandbytes datasets accelerate loralib 

In [3]:
import torch
import os
import torch.nn as nn
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, BitsAndBytesConfig

/home/ktk/anaconda3/envs/bookworm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


##### Setup QLoRA config

In [9]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant = True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dype=torch.bfloat16
)

Unused kwargs: ['bnb_4bit_compute_dype']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


##### Get baseline model

In [ ]:
model_id = "meta-llama/Meta-Llama-3-8B"
baseline_model  = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    use_cache=True,
    device_map=0,
    
)

Setup LoRA config

In [2]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

lora_config = LoraConfig(
    r=64,
    lora_alpha = 64,
    lora_dropout = 0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

In [ ]:
# Setup model for 4bit SFT
base_model = prepare_model_for_kbit_training(baseline_model)
q_model = get_peft_model(base_model, lora_config)